# Llama 4: Using the Multilingual capabilities for real-world use cases

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

This guide is designed to assist you in leveraging Llama 3.1's multilingual / translation capabilities with Bedrock Converse API for practical use cases for scenarios where English may not be needed. Our goal is to demonstrate how Llama 3.1 can be used for native lanugages to cover a broader expansion of regions globally. This notebook will provide a strong foundation for creating multilingual / translation applications.

We will begin by reviewing how to leverage the Amazon Bedrock Converse API. Once we are familiarized with the Converse API, we will generate some sample prompts to be used to explore Llama 3.1's new capability to address multilingual scenarios.

## Llama 4: Advanced AI models for Enterprise Applications

Llama 4 is a powerful family of large language models developed by Meta, specifically designed for enterprise applications. Available in two variants—Scout and Maverick—these models represent significant advancements in AI capabilities. Llama 4 has the following Key Features and Capabilities: 

- Multimodal Understanding: Accepts text plus up to 5 images as input, delivering text responses with sophisticated visual comprehension
- Extensive Multilingual Support: Proficient in Arabic, English, French, German, Hindi, Indonesian, Italian, Portuguese, Spanish, Tagalog, Thai, and Vietnamese (image understanding is English-only)
- Mixture-of-Experts Architecture: Utilizes an innovative design where only 17B parameters are active during computation, despite larger total parameter counts
Enterprise-Ready Applications: Optimized for coding tasks, tool-calling, multimodal understanding, and powering agentic systems

## Model Comparison
| Feature | Llama 4 Scout | Llama 4 Maverick |
|---------|---------------|------------------|
| Active parameters | 17B | 17B |
| Total parameters | 109B | 400B |
| Number of experts | 16 | 128 |
| Single GPU capability | Yes (INT4-quantized on H100) | No |
| Maximum context length | 10M tokens | 1M tokens |

These models represent a significant advancement over previous Llama versions, with substantially improved reasoning capabilities, multimodal understanding, and performance on complex enterprise tasks. The mixture-of-experts architecture allows for impressive capabilities while maintaining efficient computation during inference.

## Multilingual Capabilities with Llama 4

Llama 4 represents a significant advancement in multilingual AI capabilities, leveraging a novel training technique called MetaP to enhance its linguistic range and performance. This innovative approach allows for the precise calibration of critical model hyper-parameters, including per-layer learning rates and initialization scales, with remarkable consistency across varying batch sizes, model architectures, and training volumes.


The multilingual capabilities of Llama 4 are particularly impressive, having been pre-trained on an extensive corpus spanning 200 languages. More than half of these languages are represented by over 1 billion tokens each, resulting in a training dataset with 10 times more multilingual content than its predecessor, Llama 3. This extensive linguistic coverage makes Llama 4 exceptionally well-suited for global applications requiring cross-lingual understanding and generation.


Llama 4 Maverick, the flagship model in this series, features a sophisticated mixture-of-experts architecture with 17 billion active parameters distributed across 128 experts, totaling 400 billion parameters. Despite this complexity, it delivers superior performance at a more competitive price point compared to Llama 3.3 70B. Independent benchmarks confirm Llama 4 Maverick's excellence across multiple domains, outperforming comparable models like GPT-4o and Gemini 2.0 in coding, reasoning, multilingual tasks, long-context handling, and image processing. It even rivals the much larger DeepSeek v3.1 model in specialized coding and reasoning challenges.

In this notebook, we will go over a few practical real-world use cases (add one more sentences)

## Prerequisites:
- AWS SDK for Python (Boto3): Ensure you have the Boto3 library installed.
- Amazon Bedrock Model ID: Obtain the model ID for the conversational AI model you wish to use.
- AWS Credentials: Configure your AWS credentials to authenticate API requests.

## Import necessary libraries

In the next cell, we will set up our environment by importing the boto3 library and necessary libraries that we will need to confiure our bedrock client to invoke our llama 4 model within the Amazon Bedrock Converse API.

In [47]:
import boto3
from boto3 import client
from botocore.config import Config
import json
import re
import logging
from botocore.exceptions import ClientError
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [19]:
config = Config(read_timeout=2000)
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-east-1", config=config)

## Instantiate llama 4 model 

In [20]:
meta_llama_4_maverick = 'us.meta.llama4-maverick-17b-instruct-v1:0'
meta_llama_4_scout = 'us.meta.llama4-scout-17b-instruct-v1:0'

model_id = meta_llama_4_scout

## Prompting with Amazon Bedrock Converse API

The Amazon Bedrock Converse API can be used to invoke large language models on Amazon Bedrock by extracting away the special stop tokens and facilitate the creation of conversational applications by enabling the exchange of messages between users and Amazon Bedrock models.

**Key features include:**
- **Consistent Interface:**
The Converse API provides a uniform interface that works across all Amazon Bedrock models supporting messages. This consistency allows developers to write code once and use it with different models without needing to adjust for model-specific differences.

- **Turn-Based Conversations:**
The API supports multi-turn conversations, where a series of messages are exchanged between the user (acting as the user role) and the model (acting as the assistant role). This enables the development of chatbots and other conversational agents that can maintain context over multiple interactions.

- **Tool Use (Function Calling):**
The Converse API supports tool use, allowing models to request the invocation of external functions or tools. This is particularly useful for tasks that require interaction with external APIs or services. The model generates a JSON structure with the necessary parameters, which the calling application then uses to invoke the specified tool

## Prompting with the Bedrock Converse API

After setting your model ID as llama 4 scout model, to start interfacing with the converse API - you can put the prompt input inside the content section.

Listed below is an example of how you can change / assign the different roles and add input to the role.

#### Prompting with Converse API on Bedrock

```
{
    "role": "user | assistant",
    "content": [
        {
            "text": "string"
        }
    ]
}
```

In the next cell, we will use llama 4 to generate a few questions that we will use to be allow us to have a conversation in French.

In [31]:
# Prepare your payload with the messages format
payload = {
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": "You are a helpful AI assistant."
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "text": "What are three key benefits of large language models for businesses?"
                }
            ]
        }
    ]

}

In [32]:
response = bedrock_client.converse(
    modelId=model_id,
    messages=payload["messages"],  # Wrap the message in a list
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0,
        "topP": .5
    }
)

print(response["output"]["message"]["content"][0]["text"])

Large language models (LLMs) have been increasingly adopted by businesses due to their potential to drive efficiency, innovation, and growth. Here are three key benefits of LLMs for businesses:

1. **Improved Customer Experience**: LLMs can be used to power chatbots, virtual assistants, and other customer-facing applications, enabling businesses to provide 24/7 support, respond to customer inquiries, and offer personalized recommendations. By automating routine customer interactions, businesses can free up human customer support agents to focus on more complex issues, leading to improved customer satisfaction and loyalty.

2. **Enhanced Content Generation and Productivity**: LLMs can generate high-quality content, such as product descriptions, blog posts, and social media updates, at scale and with minimal human input. This can help businesses to reduce content creation costs, improve content consistency, and accelerate their content marketing efforts. Additionally, LLMs can assist wit

In the cell above, we can see that Llama 4 was able to successfully generate some sample questions in English. We will use these questions later on in the notebook to prompt Llama 3.1 to have a full conversation in French. But first, let's go into a few of the multilingual translation basics.

## Multi-lingual use case capabilities: Translation

Llama 4 supports a variety of languages, allowing developers to build applications and services that cater to users from diverse linguistic backgrounds by offering multilingual capabilities that can be leveraged to create localized user interfaces and provide language-specific content. Specifically, Llama 4 supports languages such as English, French, German, Spanish, Italian, Portuguese, Hindi and Thai.

For translating telephone conversations into different languages like Hindi, Arabic, Korean, and Japanese, Llama 4 can be utilized effectively. In the next cell, we will demonstrate how a bank teller and customer conversation that is held in English can be translated to various languages.

In [34]:
conversation = """
Customer: "Hi, I recently opened a new account with your bank and I love the online banking features! Can you guide me on how to set up different alerts for each type of transaction I make?"
Teller: "Hello! I'm glad to hear you're enjoying our services. Setting up transaction alerts is simple. You can customize them through our online banking portal. Let me walk you through the process."
Customer: "I'm trying to use the mobile banking app, but it doesn't seem to be registering my login attempts. Could you help me figure out what might be going wrong?"
Teller: "Of course! Let's troubleshoot this together. First, ensure that your username and password are correct. If you're still having issues, try resetting your password or updating the app to the latest version." 
Customer: "I'm considering opening a savings account and I'm curious about the interest rates. What options are available and what are their main differences?"
Teller: "Great question! We offer several savings account options with varying interest rates. Our standard savings account has a competitive rate, while our high-yield account offers a higher rate for larger balances. I can provide more details on each."
Customer: "I wanted to report a small issue where my debit card was a bit sticky. However, your quick-start guide was super helpful and I fixed it easily by following the cleaning tips. Just thought you might want to know!"
Teller: "Thanks for letting us know. I'm glad the guide was helpful. If you encounter any more issues, feel free to reach out, and we'll be happy to assist further." 
Customer: "My new debit card stopped working within a week of receiving it. None of the transactions go through, and I need a solution or a replacement as soon as possible."
Teller: "I'm sorry to hear that. Let's resolve this quickly. Please check if the card is activated and try using a different ATM. If it still doesn't work, we'll issue a replacement card right away." 
Customer: "I've noticed that the numbers on my debit card are starting to fade after several months of use. Is this covered by the warranty?"
Teller: "Yes, card wear and tear is covered. Please bring your card to our branch, and we'll assist you with a replacement at no charge." 
Customer: "I had an issue where my account settings would reset every time I logged in. I figured out it was due to a browser conflict and resolved it by clearing my cache. Just wanted to ask if there are any new updates coming soon?"
Teller: "Good job on resolving the issue. We do roll out updates regularly to improve our online services. Keep an eye on our website or subscribe to our newsletter for the latest updates." 
Customer: "I've been having trouble with the online banking portal not saving my preferences, and it's starting to get frustrating. What can be done to ensure my settings are saved permanently?"
Teller: "I understand your frustration. Make sure you're using the latest version of your browser. Also, try adjusting your browser settings to allow cookies, which can help save your preferences."
"""

Here we will allow llama 4 to translate each of the following transcripted calls into different langauges by creating different prompts for Hindi, Spanish, german and italian. These prompts will all be used to ingest the conversation and translate into the native given language.

In [35]:
#prompts for language tasks/ translation

hindi_prompt = f"""
Translate the following conversation into Hindi. Your responses must be numbered, only in Hindi, and must adhere to only translating the conversation.

{conversation}
"""

spanish_prompt = f"""
Translate the following conversation into Spanish. Your responses must be numbered, only in Spanish, and must adhere to only translating the conversation.

{conversation}
"""

german_prompt = f"""
Translate the following conversation into German. Your responses must be numbered, only in German, and must adhere to only translating the conversation.

{conversation}
"""

italian_prompt = f"""
Translate the following conversation into Italian. Your responses must be numbered, only in Italian, and must adhere to only translating the conversation.

{conversation}
"""

In [48]:
def converse(prompt, inference_config):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    response = bedrock_client.converse(
        messages=messages,
        modelId=model_id,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    print(generated_text)
    return generated_text, response

inference_config = {"temperature": 0.0, "maxTokens": 2048, "topP": 0.1}

#hindi prompt with llama 4
generated_text, response = converse(hindi_prompt, inference_config)

1. ग्राहक: "नमस्ते, मैंने हाल ही में आपके बैंक में एक नया खाता खोला है और मुझे ऑनलाइन बैंकिंग सुविधाएं बहुत पसंद हैं! क्या आप मुझे बता सकते हैं कि मैं अपने प्रत्येक लेनदेन के लिए अलग-अलग अलर्ट कैसे सेट कर सकता हूं?"
2. टेलर: "नमस्ते! मुझे यह जानकर खुशी हुई कि आप हमारी सेवाओं का आनंद ले रहे हैं। लेनदेन अलर्ट सेट करना आसान है। आप हमारे ऑनलाइन बैंकिंग पोर्टल के माध्यम से उन्हें अनुकूलित कर सकते हैं। मैं आपको इस प्रक्रिया के बारे में बताऊंगा।"
3. ग्राहक: "मैं मोबाइल बैंकिंग ऐप का उपयोग करने की कोशिश कर रहा हूं, लेकिन यह मेरे लॉगिन प्रयासों को पंजीकृत नहीं कर रहा है। क्या आप मुझे यह पता लगाने में मदद कर सकते हैं कि क्या गलत हो रहा है?"
4. टेलर: "बिल्कुल! आइए इसे एक साथ हल करें। सबसे पहले, सुनिश्चित करें कि आपका उपयोगकर्ता नाम और पासवर्ड सही है। यदि आपको अभी भी समस्याएं आ रही हैं, तो अपना पासवर्ड रीसेट करने या ऐप को नवीनतम संस्करण में अपडेट करने का प्रयास करें।"
5. ग्राहक: "मैं एक बचत खाता खोलने पर विचार कर रहा हूं और मैं ब्याज दरों के बारे में उत्सुक हूं। क्या विकल्प उपलब्ध हैं और उनके मुख्

In [37]:
#spanish prompt with llama 4
generated_text, response = converse(spanish_prompt, inference_config)

1. Cliente: "Hola, recientemente abrí una nueva cuenta con su banco y me encantan las características de la banca en línea. ¿Puede guiarme sobre cómo configurar diferentes alertas para cada tipo de transacción que realizo?"
2. Cajero: "Hola. Me alegra saber que está disfrutando de nuestros servicios. Configurar alertas de transacciones es sencillo. Puede personalizarlas a través de nuestro portal de banca en línea. Déjeme explicarle el proceso."
3. Cliente: "Estoy intentando usar la aplicación de banca móvil, pero no parece estar registrando mis intentos de inicio de sesión. ¿Podría ayudarme a descubrir qué podría estar saliendo mal?"
4. Cajero: "Por supuesto. Vamos a solucionar esto juntos. Primero, asegúrese de que su nombre de usuario y contraseña sean correctos. Si todavía tiene problemas, intente restablecer su contraseña o actualizar la aplicación a la última versión."
5. Cliente: "Estoy considerando abrir una cuenta de ahorros y estoy curioso sobre las tasas de interés. ¿Qué opc

In [38]:
#german prompt with llama 4
generated_text, response = converse(german_prompt, inference_config)

1. Kunde: "Hallo, ich habe kürzlich ein neues Konto bei Ihrer Bank eröffnet und liebe die Online-Banking-Funktionen! Können Sie mir helfen, verschiedene Benachrichtigungen für jede Art von Transaktion, die ich durchführe, einzurichten?"
2. Bankangestellter: "Hallo! Ich bin froh, dass Sie unsere Dienstleistungen genießen. Das Einrichten von Transaktionsbenachrichtigungen ist einfach. Sie können diese über unser Online-Banking-Portal anpassen. Lassen Sie mich Sie durch den Prozess führen."
3. Kunde: "Ich versuche, die Mobile-Banking-App zu verwenden, aber es scheint, als ob meine Anmeldeversuche nicht registriert werden. Könnten Sie mir helfen, herauszufinden, was möglicherweise schief läuft?"
4. Bankangestellter: "Natürlich! Lassen Sie uns dies gemeinsam beheben. Stellen Sie zunächst sicher, dass Ihr Benutzername und Passwort korrekt sind. Wenn Sie weiterhin Probleme haben, versuchen Sie, Ihr Passwort zurückzusetzen oder die App auf die neueste Version zu aktualisieren."
5. Kunde: "Ich 

In [49]:
# italian prompt with llama 4
generated_text_italian, response = converse(italian_prompt, inference_config)

1. Cliente: "Ciao, ho recentemente aperto un nuovo conto con la vostra banca e adoro le funzionalità di banking online! Potete guidarmi su come impostare diversi avvisi per ogni tipo di transazione che faccio?"
2. Bancario: "Ciao! Sono felice di sentire che stai godendo dei nostri servizi. Impostare avvisi di transazione è semplice. Puoi personalizzarli attraverso il nostro portale di banking online. Lasciami guidarti attraverso il processo."
3. Cliente: "Sto cercando di utilizzare l'app di mobile banking, ma non sembra registrare i miei tentativi di accesso. Potreste aiutarmi a capire cosa potrebbe andare storto?"
4. Bancario: "Certo! Risolviamo questo insieme. Innanzitutto, assicurati che il tuo nome utente e la password siano corretti. Se continui ad avere problemi, prova a reimpostare la password o ad aggiornare l'app all'ultima versione."
5. Cliente: "Sto considerando di aprire un conto di risparmio e sono curioso riguardo ai tassi di interesse. Quali opzioni sono disponibili e qu

## Translation evaluation with Larger Maverick model

In [40]:
eval_model = meta_llama_4_maverick

In [50]:
# Evaluate the generated text using the maverick model
eval_prompt = f"""The {conversation} text was translated from English to italian 
by converting the conversation into {generated_text_italian} text, is this an accurate translation? 
Highlight any useful translations in English that could make the new language more accurate."""

In [42]:
def converse(prompt, inference_config):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    response = bedrock_client.converse(
        messages=messages,
        modelId=eval_model,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    print(generated_text)
    return generated_text, response

inference_config = {"temperature": 0.0, "maxTokens": 2048, "topP": 0.1}


generated_text, response = converse(eval_prompt, inference_config)

The provided Italian translation is generally accurate. However, there are a few minor adjustments that could be made to improve its accuracy and naturalness.

1. Cliente: "Ciao, ho recentemente aperto un nuovo conto con la vostra banca e adoro le funzionalità di banking online! Potete guidarmi su come impostare diversi avvisi per ogni tipo di transazione che faccio?"
   - The translation is accurate. "Banking online" could be translated to "online banking" as "servizi di online banking" for a more natural tone.

2. Cliente: "Sto cercando di utilizzare l'app di mobile banking, ma non sembra registrare i miei tentativi di accesso."
   - "Mobile banking app" is more naturally translated as "app di banking mobile" or simply "l'app mobile" in informal contexts, but "l'app di mobile banking" is acceptable.

3. Bancario: "Certo! Risolviamo questo insieme."
   - A more natural translation could be "Certo! Vediamo di risolvere insieme" to convey a sense of collaborative troubleshooting.

4. Cl

## Multilingual Chatbot functionality

LLama 4's multilingual capabilities not only allow developers to translate content from one language to another, but also enables for multilingual chatbots. This is useful for scenarios where developers want to expand globally and create a chatbot to be used to speak in entirely different native languages other than english.

If you remember earlier in the notebook, we generated some sample questions that a tourist may have when they look to travel Paris. In the next cell, we will store the questions (in english) as a prompt, have Llama 4 translate the questions in French, and answer the questions in French.

In [43]:
questions = f"""

{formatted_text}
"""

In [44]:
# Define the message to translate and answer the questions in French
translated_convo = {
    "role": "user",
    "content": [
        {"text": f"Please translate the following questions into French. Once you translate the questions, please answer the questions in French as well.\n\n{questions}"}
    ]
}

In [45]:
# Send the request to the model
response = bedrock_client.converse(
    modelId=model_id,
    messages=[translated_convo],  # Wrap the message in a list
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0,
        "topP": 0.5
    }
)

In [46]:
# Extract the 'text' content from the response
text_content = response['output']['message']['content'][0]['text']

# Replace newline escape sequences ('\\n') with actual newline characters
formatted_text_2 = text_content.replace('\\n', '\n')

# Print the formatted text
print(formatted_text_2)

Here are the translations of the three key benefits into questions in French, followed by the answers:

**Questions :**
1. Comment les grands modèles de langage (LLM) améliorent-ils l'expérience client ?
2. De quelle manière les LLM améliorent-ils la génération de contenu et la productivité ?
3. Comment les LLM contribuent-ils à une prise de décision basée sur les données ?

**Réponses :**
1. Les LLM améliorent l'expérience client en alimentant les chatbots, les assistants virtuels et d'autres applications destinées aux clients, permettant aux entreprises de fournir un soutien 24/7, de répondre aux demandes des clients et d'offrir des recommandations personnalisées. En automatisant les interactions routinières avec les clients, les entreprises peuvent libérer les agents de soutien client humains pour se concentrer sur des problèmes plus complexes, ce qui conduit à une satisfaction et une fidélité accrues des clients.

2. Les LLM améliorent la génération de contenu et la productivité en

## Conclusion

In conclusion, Llama 4 represents a significant advancement in multilingual capabilities, enabling the creation of sophisticated multilingual chatbots and translation services. The model supports a wide array of languages, including English, French, German, Hindi, Italian, Portuguese, Spanish, and Thai, making it a versatile tool for global applications. Its ability to handle complex tasks with a context length of up to 3.5M tokens allows it to manage long conversations and provide accurate translations, which are crucial for developing multilingual chatbots.

Overall, Llama 4's multilingual capabilities not only facilitate effective translation across different languages but also enable the development of intelligent conversational agents that can interact seamlessly with users worldwide. This makes it an invaluable resource for businesses and developers looking to expand their reach in diverse linguistic markets.

---
## Distributors
- Amazon Web Services
- Meta